In [1]:
import pandas as pd
pd.set_option('display.max_columns', 600)

#ens_id <-> gene name converter
import mygene
mg = mygene.MyGeneInfo()

from scipy import stats
import numpy as np

In [2]:
#using UCSC Xena data
cesc1 = pd.read_csv('./TCGA-CESC.GDC_phenotype.tsv', sep='\t')
cesc2 = pd.read_csv('./TCGA-CESC.htseq_fpkm-uq.tsv', sep='\t', header=None)

cesc2[0] = cesc2[0].apply(lambda x: x.split('.')[0])
cesc2 = cesc2.T
cesc2.columns = cesc2.iloc[0,:]

C:\Users\bioinfo205_01\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249

In [3]:
#target gene *46*
gene = ['A4GALT','ASAH1','NAAA','B4GALT6','BECN1','BNIP3','CERK','DEGS1',
        'S1PR1','LPAR1','LPAR2','S1PR2','S1PR4','KDSR','GAL3ST1','GALC','B4GALNT1',
        'GBA','HPGD','CERS1','CERS2','CERS3','CERS4','CERS5','CERS6','LCT',
        'LOC51190','NSMAF','SFTPB','SLC26A10','SMPD1','SMPD2','SMPDL3A','SMPDL3B',
        'SPHK1','SPHK2','SPTLC1','SPTLC2','ST3GAL5','ST6GALNAC5','ST8SIA1',
        'ST8SIA3','UGCG','UGGT1','UGGT2','UGT8']

#ens_id serching
out = mg.querymany(gene, scopes='symbol', fields='ensembl.gene', species='human')

#gene3: gene - ens_id df
gene2 = pd.DataFrame(['enpty'])

for i in range(0, len(out)):
    try:
        gene2.loc[i] = out[i]['ensembl']['gene']
    except KeyError:
        gene2.loc[i] = 'unknown'+str(i)
    except TypeError:
        gene2.loc[i] = out[i]['ensembl'][0]['gene']

gene = pd.DataFrame(gene, columns=['gene'])
        
gene3 = gene
gene3['ens_id'] = gene2

querying 1-46...done.
Finished.
1 input query terms found no hit:
	['LOC51190']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [4]:
#clinical data - RNAseq data merge
cesc = pd.merge(cesc1, cesc2,
             how='inner',
             left_on='submitter_id.samples',
             right_on='Ensembl_ID',
             sort=True,
             indicator=False)

In [5]:
#paired data masking
m = cesc['submitter_id.samples'].str.contains('-11') == True

#'TCGA-FU-A3EO-01A' - 'TCGA-FU-A3EO-01A'
#'TCGA-HM-A3JJ-01A' - 'TCGA-HM-A3JJ-11A'
#'TCGA-MY-A5BF-01A' - 'TCGA-MY-A5BF-11A'
m2 = cesc['submitter_id.samples'] == 'TCGA-FU-A3EO-01A'
m3 = cesc['submitter_id.samples'] == 'TCGA-HM-A3JJ-01A'
m4 = cesc['submitter_id.samples'] == 'TCGA-MY-A5BF-01A'

temp1 = cesc[m2]
temp2 = cesc[m3]
temp3 = cesc[m4]

tumor = pd.concat([temp1, temp2, temp3]).reset_index()
n_tumor = cesc[m].reset_index()

In [6]:
#target gene masking
tumor2 = tumor.T
tumor2['index1'] = tumor2.index

tumor2 = pd.merge(tumor2, gene3,
             how='inner',
             left_on='index1',
             right_on='ens_id',
             sort=False,
             indicator=False)

tumor2 = tumor2.T
tumor2.columns = tumor2.iloc[-1,:]


n_tumor2 = n_tumor.T

n_tumor2['index1'] = n_tumor2.index

n_tumor2 = pd.merge(n_tumor2, gene3,
             how='inner',
             left_on='index1',
             right_on='ens_id',
             sort=False,
             indicator=False)

n_tumor2 = n_tumor2.T
n_tumor2.columns = n_tumor2.iloc[-1,:]

In [7]:
tumor3 = tumor2.iloc[0:3]
n_tumor3 = n_tumor2.iloc[0:3]

for i in range(0,(tumor3.shape[1])):
    tumor3.iloc[:,i] = pd.to_numeric(tumor3.iloc[:,i])
    n_tumor3.iloc[:,i] = pd.to_numeric(n_tumor3.iloc[:,i])

#shapiro-wilk test. p<.05  >>> Wilcoxon test
print(stats.shapiro(tumor3))
print(stats.shapiro(n_tumor3))

C:\Users\bioinfo205_01\Anaconda3\lib\site-packages\pandas\core\indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\bioinfo205_01\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(0.7953522205352783, 1.9015444600867237e-12)
(0.8101568222045898, 6.219632014276e-12)


In [8]:
#Wilcoxon’s signed-ranks test
#res: [result, p-value] df
res = pd.DataFrame()

for i in range (tumor3.shape[1]):
    temp = stats.wilcoxon(pd.to_numeric(tumor2.iloc[0:3,i]), pd.to_numeric(n_tumor3.iloc[0:3,i]))
    res.insert(i, i, [tumor2.iloc[3,i], tumor2.iloc[4,i], temp[0], temp[1]])

res.columns = res.iloc[0,:]
res = res.iloc[1:]
res.index = ['gene', 'statistic', 'p-val']

C:\Users\bioinfo205_01\Anaconda3\lib\site-packages\scipy\stats\morestats.py:2388: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


In [9]:
#data output
tumor.to_csv('./tumor_sample.csv', index=False)
n_tumor.to_csv('./normal_sample.csv', index=False)
res.to_csv('./result.csv')

In [10]:
#Wilcoxon’s rank-sum test. uning mean value

tumor4 = tumor2

temp = []

for i in range(0,(tumor4.shape[1])):
    temp.append(np.mean(pd.to_numeric(tumor4.iloc[0:3,i])))

temp = pd.DataFrame(temp).T
temp.columns=tumor4.columns

tumor4 = tumor4.append(temp)

n_tumor4 = n_tumor2

temp = []

for i in range(0,(n_tumor4.shape[1])):
    temp.append(np.mean(pd.to_numeric(n_tumor4.iloc[0:3,i])))

temp = pd.DataFrame(temp).T
temp.columns=n_tumor4.columns

n_tumor4 = n_tumor4.append(temp)

stats.ranksums(pd.to_numeric(tumor4.iloc[-1]), pd.to_numeric(n_tumor4.iloc[-1]))

RanksumsResult(statistic=-0.5527744798375062, pvalue=0.5804178455954285)

In [11]:
#shapiro-wilk test. p<.05  >>> Wilcoxon test
print(stats.shapiro(tumor4.iloc[-1]))
print(stats.shapiro(n_tumor4.iloc[-1]))

(0.8136442303657532, 4.810991413251031e-06)
(0.8070154786109924, 3.404478093216312e-06)
